In [2]:
from http.server import BaseHTTPRequestHandler, HTTPServer
import logging
import requests
import urllib.request
import json
import numpy as np
from tensorflow.keras.models import load_model
from gensim.models import FastText
from konlpy.tag import Okt
import pandas as pd
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer
import sys
import boto3
from bs4 import BeautifulSoup 
from keras.preprocessing import sequence

MAX_NB_WORDS = 100000
kakao_model = load_model('kakao_LSTM_model.h5')
SNU_model = load_model('SNU_LSTM_Model.h5')

# Vectorizer의 argument인 tokenizer에 KoNLPy의 pos 함수로 대체.
class MyTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger

    def __call__(self, sent):
        pos = self.tagger.pos(sent)
        clean_words = []  # 정제된 단어 리스트
        for word in pos:
            # word[1]은 품사를 의미하며, 여기서는 조사, 문장기호, 접두사, Foreign('\n'을 빼주기 위함)인 것은 제외시킴.
            if word[1] not in ['Josa', 'Punctuation', 'Suffix', 'Foreign']:
                if len(word[0]) >= 2:  # 한 글자인 단어들도 의미가 없는 경우가 많으므로 일단 제외.
                    clean_words.append(word[0])
        return clean_words


my_Tokenizer = MyTokenizer(Okt())

/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


Extension horovod.torch has not been built: /home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/horovod/torch/mpi_lib/_mpi_lib.cpython-37m-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still avaiable.
[2021-08-05 05:35:35.049 ip-172-31-2-138:3111 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-08-05 05:35:35.069 ip-172-31-2-138:3111 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")

('ratings_train.txt', <http.client.HTTPMessage at 0x7faebc3c7790>)

In [4]:
if sys.version_info[0] < 3:
    from io import StringIO # Python 2.x
else:
    from io import StringIO # Python 3.x

# get your credentials from environment variables
aws_id = ''
aws_secret = ''

client = boto3.client('s3', aws_access_key_id=aws_id,
        aws_secret_access_key=aws_secret)

bucket_name = 'snucsv'

object_key = 'SNU_Data_1200.csv'
csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
body = csv_obj['Body']
csv_string = body.read().decode('cp949')

# load data
train_df = pd.read_csv(StringIO(csv_string))
label_names = ["label"]
y_train = train_df[label_names].values

train_df['doc_len'] = train_df['document'].apply(lambda words: len(words.split(" ")))
max_seq_len = np.round(train_df['doc_len'].mean() + train_df['doc_len'].std()).astype(int)
train_df

,document,label,doc_len
0,현재 AI기술은 불완전하기 때문에 가짜뉴스 완전히 걸러낼 수 없다,1,9
1,"‘정치 세대교체’ 바람 일으킨 이준석, 대선엔 출마 못한다",1,8
2,액상형 전자담배는 니코틴이 함유되지 않아 담배가 아니다,1,7
3,한국 백신 접종률 세계 100위권 이하다,1,6
4,광역도시 없는 지역권은 실제 수요와 관계없이 획일적으로 국가교통망 정책에서 소외되고 있다,1,11
...,...,...,...
1219,""" ‘당·정·청 전원회의’는 '운동권 용어'다""?",0,5
1220,"""늘어가는 해양사고, 전북 관내 잠수구조인력 전무(全無)""?",0,6
1221,"""청와대 하루 평균 업무추진비, 이명박 768만원, 박근혜 814만원, 문재인 55만원""",0,10
1222,"국민연금공단 기금운용본부가 전주로 이전한 후, 해외투자자가 국민연금을 '패싱'한다",0,8


**SNU - 단어 벡터화 및 패딩**

In [5]:
raw_docs_train = train_df['document'].tolist()
num_classes = len(label_names)
print(num_classes)
processed_docs_train = []

for doc in raw_docs_train:
    tokens = my_Tokenizer(doc)
    processed_docs_train.append(tokens)

tokenizer_SNU = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
tokenizer_SNU.fit_on_texts(processed_docs_train)
word_seq_train = tokenizer_SNU.texts_to_sequences(processed_docs_train)
word_index = tokenizer_SNU.word_index
print('dictionary size :', len(word_index))

word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=max_seq_len)
word_seq_train

1
dictionary size : 3999


array([[ 503, 1459,  504, ..., 1461, 1462,    8],
       [   0,   72,  761, ...,   51,  360,    5],
       [   0,    0, 1466, ...,  506,  763,   41],
       ...,
       [ 649,  177,  381, ..., 3992,   13, 3993],
       [3994, 3995, 3996, ...,  248, 3998,    5],
       [   0,    0,    0, ..., 1401, 3999,  496]], dtype=int32)

**kakao - 단어 벡터화 및 패딩**

In [6]:
train_data = pd.read_table('ratings_train.txt')
train_data.drop_duplicates(subset=['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
train_data['document'] = train_data['document'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
train_data['document'].replace('', np.nan, inplace=True)
train_data = train_data.dropna(how = 'any')

stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
okt = Okt()
max_len = 30

X_train = []
for sentence in train_data['document']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)
    
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트

for key, value in tokenizer.word_counts.items():
    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1

vocab_size = total_cnt - rare_cnt + 1

tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)

/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [7]:
response = '카카오톡 유료화 서비스 시행하나? 어쩌고 저쩌고 어쩌구 저쩌구 어쩌코 저쩌코 어쩌쿠 저쩌쿠'
response = okt.morphs(response, stem=True) #토큰화
response = [word for word in response if not word in stopwords] #불용어 제거
encoded = tokenizer.texts_to_sequences([response]) # 정수 인코딩
pad_new = pad_sequences(encoded, maxlen = max_len) #패딩
score = float(kakao_model.predict(pad_new))
result = ''
if(score > 0.8 or score < 0.2):
    result = "객관성이 매우 낮습니다"
elif(score > 0.65 or score < 0.35):
    result = "객관성이 낮습니다"
else:
    result = "객관성이 높습니다"

In [8]:
url = 'https://jub0t0f0nc.execute-api.ap-northeast-2.amazonaws.com/default/Python_jupyter_test'

class S(BaseHTTPRequestHandler):
    def _set_response(self):
        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.end_headers()

    def do_GET(self):
        logging.info("GET request,\nPath: %s\nHeaders:\n%s\n", str(self.path), str(self.headers))
        self._set_response()
        self.wfile.write("GET request for {}".format(self.path).encode('utf-8'))

    def do_POST(self):
        content_length = int(self.headers['Content-Length']) # <--- Gets the size of data
        post_data = self.rfile.read(content_length) # <--- Gets the data itself
        logging.info("POST request,\nPath: %s\nHeaders:\n%s\n\nBody:\n%s\n",
                str(self.path), str(self.headers), post_data.decode('utf-8'))
        
        response = str(post_data.decode('utf-8'))
        response_list = []
        response_list.append(response)
        response1 = ['카카오톡 유료화 서비스 시행하나? 어쩌고 저쩌고 어쩌구 저쩌구 어쩌코 저쩌코 어쩌쿠 저쩌쿠']
        print(response)
            
# ===========================================================================================================================
# 단어 토큰화 및 빈도순 정렬 후 추출

        tfidf_Vectorizer = TfidfVectorizer(tokenizer=my_Tokenizer, min_df=1) # df 값(단어가 몇 문장들에서 등장하였는지)을 최소 'min_df' 값으로 설정.
        X = tfidf_Vectorizer.fit_transform(response_list).toarray()
#         print(X.shape)    # X(2차원 배열)의 행,열 수를 출력.
#         print(tfidf_Vectorizer.vocabulary_)   # 각 단어들이 배열 X에서 몇번째 열(인덱스 값)에 해당하는지 출력.


        #pandas를 활용하여 각 단어들의 각 문장에서의 tf-idf 값들을 모두 더하고, 내림차순으로 정렬하여 상위 n개 출력
        count = X.sum(axis=0)    # 2차원 배열 X에서 각 열을 기준으로 합을 구함. (각 단어들의 '최종' tf-idf 값으로 간주.)
        word_count = pd.DataFrame({
            '단어' : tfidf_Vectorizer.get_feature_names(),
            '빈도' : count.flat
        })
        sorted_df = word_count.sort_values('빈도', ascending=False)
        print(sorted_df.head(10), "\n")

        word_ = list(np.array(sorted_df['단어'].tolist())) 
        
# ===========================================================================================================================
# MySQL 데이터베이스에 응답데이터, 시간 저장

        now = datetime.datetime.now()
        Korea_DatetimeNow = now + datetime.timedelta(hours = 9)
        DatetimeNow = Korea_DatetimeNow.strftime('%Y-%m-%d %H:%M:%S')
        
        query = "INSERT INTO factcheck.kakao_data (kakao_data, date) VALUES(%s, %s)"
        val = (response, DatetimeNow)
        cursor.execute(query, val)
        conn.commit()
        
# ===========================================================================================================================
# MySQL 데이터베이스에 키워드 랭킹

        rank_list = []

        cursor.execute('SELECT keyword FROM keyword_rank')
        row = cursor.fetchall()

        for i in range(len(row)):
            rank_list.append(row[i][0])
            
        for i in range(0,2):
            if word_[i] in rank_list:
                query = "UPDATE keyword_rank SET count = count + 1 WHERE keyword = (%s)"
                val = (word_[i])
                cursor.execute(query, val)
                conn.commit()
            else:
                query = "INSERT INTO factcheck.keyword_rank (keyword, count) VALUES(%s, 1)"
                val = (word_[i])
                cursor.execute(query, val)
                conn.commit()

            conn.close()
        
        top10_list = []
        cursor.execute("SELECT keyword FROM keyword_rank ORDER BY count DESC LIMIT 10")
        row_top = cursor.fetchall()
        for i in range(len(row_top)):
            top10_list.append(row_top[i][0])
            
# ===========================================================================================================================
# 텍스트 객관성 판단

        response_kakao = okt.morphs(response, stem=True) #토큰화
        response_kakao = [word for word in response if not word in stopwords] #불용어 제거
        encoded = tokenizer.texts_to_sequences([response_kakao]) # 정수 인코딩
        pad_new = pad_sequences(encoded, maxlen = max_len) #패딩
        score = float(kakao_model.predict(pad_new))
        result = ''
        if(score > 0.8 or score < 0.2):
            result = "객관성이 매우 낮습니다"
        elif(score > 0.65 or score < 0.35):
            result = "객관성이 낮습니다"
        else:
            result = "객관성이 높습니다"
        
# ===========================================================================================================================
# SNU 기사 검색        
        str_expr = "document.str.contains('{}', case=False)".format(word_[0]) and "document.str.contains('{}', case=False)".format(word_[1])
        df_q = train_df.query(str_expr, engine="python")
        
        title = list(np.array(df_q['document'].tolist()))
        # link = list(np.array(df_q['link'].tolist()))
        title_list2 = []
        # link_list2 = []

        for i in range(len(title)):
            title_list2.append(title[i])
        #     link_list2.append(link[i])
        
# ===========================================================================================================================
# 네이버 뉴스 검색
        
        baseurl = "https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query="

        url = baseurl + urllib.parse.quote_plus(response)

        html = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(html, 'html.parser')

        title = soup.find_all(class_ = 'news_tit')

        title_list = []
        link_list = []

        for i in title:
            title_list.append(i.attrs['title'])
            link_list.append(i.attrs['href'])

# ===========================================================================================================================
    
        tokens_response = []
        tokens_response2 = []
        score_list = []
        score_list2 = []
        
        for i in range(len(title_list)):
            tokens = my_Tokenizer(title_list[i])
            tokens_response.append(tokens)

        word_seq_response = tokenizer_SNU.texts_to_sequences(tokens_response)
        word_seq_response = sequence.pad_sequences(word_seq_response, maxlen=max_seq_len)
        
        for sco in range(len(title_list)):
            word_seq_response_to_score = word_seq_response[sco].reshape(1,max_seq_len)
            score_list.append(float(SNU_model.predict(word_seq_response_to_score)))
            
        for i in range(len(title_list2)):
            tokens2 = my_Tokenizer(title_list2[i])
            tokens_response2.append(tokens2)
        
        word_seq_response2 = tokenizer_SNU.texts_to_sequences(tokens_response2)
        word_seq_response2 = sequence.pad_sequences(word_seq_response, maxlen=max_seq_len)

        for sco2 in range(len(title_list2)):
            word_seq_response_to_score2 = word_seq_response2[sco2].reshape(1,max_seq_len)
            score_list2.append(float(SNU_model.predict(word_seq_response_to_score2)))
        
        data_ = {
            'response':response,
            'objectivity':result,
            'Naver_title':title_list,
            'Naver_link':link_list,
            'Naver_score':score_list,
            'SNU_title':title_list2,
            'SNU_score':score_list2
            }
        
#         requests.post(url_lambda, data = response.encode())
        requests.post(url_lambda, data = json.dumps(data_))
        
        print('입력 데이터 : \n', response)
        print('\n')
        print('객관성 여부 : \n', result)
        print('\n')
        print('네이버 뉴스 제목 : \n', title_list)
        print('\n')
        print('네이버 뉴스 링크 : \n', link_list)
        print('\n')
        print('네이버 뉴스 score : \n', score_list)
        print('\n')
        print('SNU 기사 제목 : \n', title_list2)
        print('\n')
#         print('SNU 기사 링크 : ', link_list2)
        print('SNU 기사 score : \n', score_list2)
        
        self._set_response()
        self.wfile.write("POST request for {}".format(self.path).encode('utf-8'))

def run(server_class=HTTPServer, handler_class=S, port=8081):
    logging.basicConfig(level=logging.INFO)
    server_address = ('', port)
    httpd = server_class(server_address, handler_class)
    logging.info('Starting httpd...\n')
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        pass
    httpd.server_close()
    logging.info('Stopping httpd...\n')

if __name__ == '__main__':
    from sys import argv

    if len(argv) == 2:
        run(port=int(argv[1]))
    else:
        run()

INFO:root:Starting httpd...

INFO:root:POST request,
Path: /
Headers:
Host: 3.37.215.72:8081
User-Agent: python-requests/2.25.1
Accept-Encoding: gzip, deflate
Accept: */*
Connection: keep-alive
Content-Length: 22



Body:
카카오톡 유료화



카카오톡 유료화
    단어        빈도
0   유료  0.707107
1  카카오  0.707107 

입력 데이터 : 
 카카오톡 유료화


객관성 여부 : 
 객관성이 높습니다


네이버 뉴스 제목 : 
 ["카카오의 콘텐츠 구독서비스, 핵심은 '큐레이션'", '"어뷰징뉴스 지겨워?" 취향저격은 카톡에서…새 \'카카오 뉴스\'에 쏠리는 눈', '"취향대로 본다"…구독 콘텐츠 강화하는 네이버-카카오', "유료화로 내몰리는 '클라우드 난민'", "[FreeView] 카카오모빌리티, '독점'과 '혁신' 사이"]


네이버 뉴스 링크 : 
 ['http://www.the-pr.co.kr/news/articleView.html?idxno=47419', 'https://www.techm.kr/news/articleView.html?idxno=86810', 'https://www.asiatime.co.kr/article/20210804500139', 'https://www.sedaily.com/NewsView/22OWSGUKYL', 'https://www.techm.kr/news/articleView.html?idxno=85957']


네이버 뉴스 score : 
 [0.503149151802063, 0.5029191970825195, 0.5038474202156067, 0.5057459473609924, 0.5059890747070312]


SNU 기사 제목 : 
 ['"카카오페이 얼굴인식, 중국으로 개인정보가 넘어간다"']


SNU 기사 score : 
 [0.503149151802063]


3.34.91.146 - - [05/Aug/2021 05:59:58] "POST / HTTP/1.1" 200 -
INFO:root:POST request,
Path: /
Headers:
Host: 3.37.215.72:8081
User-Agent: python-requests/2.25.1
Accept-Encoding: gzip, deflate
Accept: */*
Connection: keep-alive
Content-Length: 22



Body:
카카오톡 유료화



카카오톡 유료화
    단어        빈도
0   유료  0.707107
1  카카오  0.707107 

입력 데이터 : 
 카카오톡 유료화


객관성 여부 : 
 객관성이 높습니다


네이버 뉴스 제목 : 
 ["카카오의 콘텐츠 구독서비스, 핵심은 '큐레이션'", '"어뷰징뉴스 지겨워?" 취향저격은 카톡에서…새 \'카카오 뉴스\'에 쏠리는 눈', '"취향대로 본다"…구독 콘텐츠 강화하는 네이버-카카오', "유료화로 내몰리는 '클라우드 난민'", "[FreeView] 카카오모빌리티, '독점'과 '혁신' 사이"]


네이버 뉴스 링크 : 
 ['http://www.the-pr.co.kr/news/articleView.html?idxno=47419', 'https://www.techm.kr/news/articleView.html?idxno=86810', 'https://www.asiatime.co.kr/article/20210804500139', 'https://www.sedaily.com/NewsView/22OWSGUKYL', 'https://www.techm.kr/news/articleView.html?idxno=85957']


네이버 뉴스 score : 
 [0.503149151802063, 0.5029191970825195, 0.5038474202156067, 0.5057459473609924, 0.5059890747070312]


SNU 기사 제목 : 
 ['"카카오페이 얼굴인식, 중국으로 개인정보가 넘어간다"']


SNU 기사 score : 
 [0.503149151802063]


3.34.91.146 - - [05/Aug/2021 06:01:14] "POST / HTTP/1.1" 200 -
INFO:root:Stopping httpd...



In [ ]:
# !sudo lsof -i :8081
# !sudo kill -9 2658

COMMAND  PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
python  2658 root   98u  IPv4  29277      0t0  TCP *:tproxy (LISTEN)


In [22]:
import urllib.parse
import urllib.request
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer

sentences = ['카카오톡 유료화 서비스 시행하나?']

my_Tokenizer = MyTokenizer(Okt())
tfidf_Vectorizer = TfidfVectorizer(tokenizer=my_Tokenizer, min_df=1) # df 값(단어가 몇 문장들에서 등장하였는지)을 최소 'min_df' 값으로 설정.
X = tfidf_Vectorizer.fit_transform(sentences).toarray()
print(X.shape)    # X(2차원 배열)의 행,열 수를 출력.
print(tfidf_Vectorizer.vocabulary_)   # 각 단어들이 배열 X에서 몇번째 열(인덱스 값)에 해당하는지 출력.


#pandas를 활용하여 각 단어들의 각 문장에서의 tf-idf 값들을 모두 더하고, 내림차순으로 정렬하여 상위 n개 출력
count = X.sum(axis=0)    # 2차원 배열 X에서 각 열을 기준으로 합을 구함. (각 단어들의 '최종' tf-idf 값으로 간주.)
word_count = pd.DataFrame({
    '단어' : tfidf_Vectorizer.get_feature_names(),
    '빈도' : count.flat
})
sorted_df = word_count.sort_values('빈도', ascending=False)
print(sorted_df.head(10), "\n")

(1, 5)
{'카카오': 3, '유료': 2, '서비스': 0, '시행': 1, '하나': 4}
    단어        빈도
0  서비스  0.447214
1   시행  0.447214
2   유료  0.447214
3  카카오  0.447214
4   하나  0.447214 



In [9]:
word__ = ""
word_ = list(np.array(sorted_df['단어'].tolist()))
for i in range(0, 4):
    word__ += word_[i]
    word__ += " "
    
word__

'서비스 시행 유료 카카오 '

In [10]:
baseurl = "https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query="

url = baseurl + urllib.parse.quote_plus(word__)

html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html, 'html.parser')

title = soup.find_all(class_ = 'news_tit')

for i in title:
    print(i.attrs['title'])
    print(i.attrs['href'])
    print()

네이버↔카카오, 웹툰·웹소설 창작자 지원 경쟁
https://zdnet.co.kr/view/?no=20210730151339

[아!이뉴스] 정부, 유료방송 규제완화 선언…넷마블 vs 엔씨 '정면승부'
http://www.inews24.com/view/1389882

"기사 평점 낮으면 배차 혜택 못 준다" 카카오 택시 약관 변경
http://yna.kr/AKR20210623173600017?did=1195m

카카오모빌리티 서비스, 혁신이냐 독점 강화냐
https://www.seoul.co.kr/news/newsView.php?id=20210715021008&wlog_tag3=naver

